In [1]:
import json
import requests
import bs4
import string
import re
import html
import unidecode

In [2]:
TRACKS_FILE_NAME = '../data/songs.json'
LYRICS_FILE_NAME = 'songs_lyrics_5000.json'

In [3]:
with open(TRACKS_FILE_NAME) as json_file:
    tracks = json.load(json_file)

In [4]:
def clean_name(name):
    name = name.replace('-', ' ')
    name = re.sub(r'[^\w\s]','', name)
    name = unidecode.unidecode(name)
    return name

In [5]:
lyrics_list = list() # collector for lyric strings or None if no lyrics found

for index, track in enumerate(tracks[:5000]):
    artist = track['artists'][0]['name']
    artist = artist.replace('&', 'and')
    artist = clean_name(artist)
    artist_2 = re.sub(r'^the ', '', artist) # if initial round fails
    song = track['name']
    song = song.partition('-')[0]
    song = song.partition('(')[0]
    song = song.strip()
    song = clean_name(song)
    source = None
    
    # generate URLs for each website. some uses dashes in place of 
    # whitespaces, others use underscores
    urls = list()
    metro_url = 'http://metrolyrics.com/{}-lyrics-{}.html'.format(
        song.replace(' ', '-'), artist.replace(' ', '-'))
    song_url = 'http://songlyrics.com/{}/{}-lyrics/'.format(
        artist.replace(' ', '-'), song.replace(' ', '-'))
    mode_url = 'http://www.lyricsmode.com/lyrics/{}/{}/{}.html'.format(
        artist[0], artist.replace(' ', '_'), song.replace(' ', '_'))
    genius_url = 'https://genius.com/{}-{}-lyrics'.format(
        artist.replace(' ', '-'), song.replace(' ', '-'))
    urls.extend([genius_url, song_url, mode_url, metro_url])
    if artist != artist_2:
        genius_url = 'https://genius.com/{}-{}-lyrics'.format(
            artist_2.replace(' ', '-'), song.replace(' ', '-'))
        metro_url = 'http://metrolyrics.com/{}-lyrics-{}.html'.format(
                song.replace(' ', '-'), artist_2.replace(' ', '-'))
        song_url = 'http://songlyrics.com/{}/{}-lyrics/'.format(
            artist_2.replace(' ', '-'), song.replace(' ', '-'))
        mode_url = 'http://www.lyricsmode.com/lyrics/{}/{}/{}.html'.format(
            artist_2[0], artist_2.replace(' ', '_'), song.replace(' ', '_'))
        urls.extend([genius_url, song_url, mode_url, metro_url])
    
    # attempt scrape and parse sequence. lyrics collected as list of tokens.
    for url in urls:
        try:
            # request HTML and parse
            html = requests.get(url=url).content
            soup = bs4.BeautifulSoup(html)
            
            # find lyrics and pre-process
            if 'genius.com' in url:
                print(url)
                lyrics = soup\
                    .find('div', {'class': 'lyrics'})\
                    .find('p')\
                    .findAll(text=True)
                if lyrics is None:
                    raise Exception
                source = 'genius.com'
                
            # for songlyrics.com, if no lyrics found, a message is displayed
            # where lyrics would be. thus, we check for this message instead.
            if 'songlyrics' in url:
                print(url)
                lyrics = soup\
                    .find('p', {'id': 'songLyricsDiv'})\
                    .findAll(text=True)
                if 'Sorry, we have no ' in lyrics[0] or 'We do not have the lyrics' in lyrics[0]:
                    lyrics = None
                    raise Exception
                source = 'songlyrics.com'
            if 'lyricsmode' in url:
                print(url)
                lyrics = soup.find('p', {'id': 'lyrics_text'})
                if lyrics is None:
                    raise Exception
                lyrics = lyrics.text.split()
                source = 'lyricsmode.com'
            if 'metrolyrics' in url:
                print(url)
                lyrics = soup.find('div', {'id': 'lyrics-body-text'})
                if lyrics is None:
                    raise Exception
                lyrics = [line.findAll(text=True)\
                          for line in soup.findAll('p', {'class': 'verse'})]
                lyrics = [item for sublist in lyrics for item in sublist]
                source = 'metrolyrics.com'
            
            break
        except Exception as e:
            print(e)
    lyrics_list.append({'id': track['id'], 'lyrics': lyrics, 'source': source})

https://genius.com/Bee-Gees-Night-Fever-lyrics
https://genius.com/Sister-Sledge-We-Are-Family-lyrics
https://genius.com/The-Trammps-Disco-Inferno-lyrics
https://genius.com/KC-and-The-Sunshine-Band-Thats-the-Way-I-Like-It-lyrics
https://genius.com/CHIC-Le-Freak-lyrics
https://genius.com/Bee-Gees-Stayin-Alive-lyrics
https://genius.com/Earth-Wind-and-Fire-September-lyrics
https://genius.com/Madonna-Holiday-lyrics
https://genius.com/Donna-Summer-Last-Dance-lyrics
https://genius.com/CHIC-Good-Times-lyrics
https://genius.com/Sister-Sledge-Hes-the-Greatest-Dancer-lyrics
https://genius.com/Thelma-Houston-Dont-Leave-Me-This-Way-lyrics
https://genius.com/Barry-White-Cant-Get-Enough-Of-Your-Love-Babe-lyrics
https://genius.com/Donna-Summer-Hot-Stuff-lyrics
https://genius.com/Michael-Jackson-Dont-Stop-Til-You-Get-Enough-lyrics
https://genius.com/Kool-and-The-Gang-Celebration-lyrics
https://genius.com/Hues-Corporation-Rock-the-Boat-lyrics
https://genius.com/Amii-Stewart-Knock-On-Wood-lyrics
https://

https://genius.com/Chuck-Berry-Johnny-B-Goode-lyrics
https://genius.com/The-Easybeats-Friday-on-My-Mind-lyrics
https://genius.com/The-Rolling-Stones-Paint-It-Black-lyrics
https://genius.com/The-Monkees-Im-a-Believer-lyrics
https://genius.com/The-Drifters-Save-the-Last-Dance-for-Me-lyrics
https://genius.com/Connie-Francis-Stupid-Cupid-lyrics
https://genius.com/Ben-E-King-Stand-by-Me-lyrics
https://genius.com/Van-Morrison-Brown-Eyed-Girl-lyrics
https://genius.com/The-Righteous-Brothers-Unchained-Melody-lyrics
https://genius.com/The-Loved-Ones-The-Loved-One-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/The-Loved-Ones/The-Loved-One-lyrics/
https://genius.com/The-Marvelettes-Please-Mr-Postman-lyrics
https://genius.com/The-Mamas-and-The-Papas-Dream-A-Little-Dream-Of-Me-lyrics
https://genius.com/Bay-City-Rollers-Bye-Bye-Baby-lyrics
https://genius.com/The-Doobie-Brothers-Listen-to-the-Music-lyrics
https://genius.com/Bill-Withers-Lean-on-Me-lyrics
https://genius.com/Tom

https://genius.com/Scorpions-Wind-Of-Change-lyrics
https://genius.com/Iron-Maiden-Run-to-the-Hills-lyrics
https://genius.com/Status-Quo-Rockin-All-Over-The-World-lyrics
https://genius.com/The-Who-Wont-Get-Fooled-Again-lyrics
https://genius.com/The-Cranberries-Zombie-lyrics
https://genius.com/Free-All-Right-Now-lyrics
https://genius.com/Led-Zeppelin-Black-Dog-lyrics
https://genius.com/Depeche-Mode-Personal-Jesus-lyrics
https://genius.com/Metallica-Nothing-Else-Matters-lyrics
https://genius.com/Haddaway-What-Is-Love-lyrics
https://genius.com/Status-Quo-In-The-Army-Now-lyrics
https://genius.com/Peter-Gabriel-Sledgehammer-lyrics
https://genius.com/Gigi-DAgostino-LAmour-Toujours-lyrics
https://genius.com/Def-Leppard-Pour-Some-Sugar-On-Me-lyrics
https://genius.com/Dead-Or-Alive-You-Spin-Me-Round-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Dead-Or-Alive/You-Spin-Me-Round-lyrics/
https://genius.com/Fleetwood-Mac-The-Chain-lyrics
https://genius.com/Darude-Sandstorm-ly

https://genius.com/Edward-Maya-Stereo-Love-lyrics
https://genius.com/Miami-DJ-Collective-Tocas-Miracle-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Miami-DJ-Collective/Tocas-Miracle-lyrics/

http://www.lyricsmode.com/lyrics/M/Miami_DJ_Collective/Tocas_Miracle.html

http://metrolyrics.com/Tocas-Miracle-lyrics-Miami-DJ-Collective.html

https://genius.com/Dizzee-Rascal-Holiday-lyrics
https://genius.com/Romina-Johnson-Movin-Too-Fast-lyrics
https://genius.com/Urban-Cookie-Collective-The-Key-The-Secret-lyrics
https://genius.com/T2-Heartbroken-lyrics
https://genius.com/Kid-Cudi-Day-N-Night-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Kid-Cudi/Day-N-Night-lyrics/
https://genius.com/Livin-Joy-Dont-Stop-Movin-lyrics
https://genius.com/Baby-D-Let-Me-Be-Your-Fantasy-lyrics
https://genius.com/Example-Kickstarts-lyrics
https://genius.com/Dizzee-Rascal-Dance-Wiv-Me-lyrics
https://genius.com/Ian-Carey-Project-Get-Shaky-lyrics
'NoneType' object has no

http://songlyrics.com/Booka-Shade/In-White-Rooms-lyrics/

http://www.lyricsmode.com/lyrics/B/Booka_Shade/In_White_Rooms.html

http://metrolyrics.com/In-White-Rooms-lyrics-Booka-Shade.html

https://genius.com/Tiesto-Adagio-For-Strings-lyrics
https://genius.com/Ercola-Deep-At-Night-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Ercola/Deep-At-Night-lyrics/
https://genius.com/Joachim-Garraud-The-World-Is-Mine-lyrics
https://genius.com/Bellatrax-Feat-Sophia-May-I-Cant-Help-Myself-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Bellatrax-Feat-Sophia-May/I-Cant-Help-Myself-lyrics/

http://www.lyricsmode.com/lyrics/B/Bellatrax_Feat_Sophia_May/I_Cant_Help_Myself.html

http://metrolyrics.com/I-Cant-Help-Myself-lyrics-Bellatrax-Feat-Sophia-May.html

https://genius.com/Alex-Gaudino-Destination-Calabria-lyrics
https://genius.com/The-Chemical-Brothers-Hey-Boy-Hey-Girl-lyrics
https://genius.com/Freemasons-Believer-lyrics
https://genius.com/Lasgo-Gone-ly

...

https://genius.com/Henry-Gauntlet-Once-in-Royal-Davids-City-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Henry-Gauntlet/Once-in-Royal-Davids-City-lyrics/

http://www.lyricsmode.com/lyrics/H/Henry_Gauntlet/Once_in_Royal_Davids_City.html

http://metrolyrics.com/Once-in-Royal-Davids-City-lyrics-Henry-Gauntlet.html

https://genius.com/Anonymous-The-First-Nowell-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Anonymous/The-First-Nowell-lyrics/

http://www.lyricsmode.com/lyrics/A/Anonymous/The_First_Nowell.html

http://metrolyrics.com/The-First-Nowell-lyrics-Anonymous.html

https://genius.com/John-Williams-Somewhere-in-My-Memory-lyrics
https://genius.com/Simon-Mulligan-Hark-The-Herald-Angels-Sing-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Simon-Mulligan/Hark-The-Herald-Angels-Sing-lyrics/

http://www.lyricsmode.com/lyrics/S/Simon_Mulligan/Hark_The_Herald_Angels_Sing.html

http://metrolyrics.com/Hark-The-Herald-Angel

http://metrolyrics.com/The-Shepherds-Carol-lyrics-Bob-Chilcott.html

https://genius.com/Reginald-Jacques-Good-King-Wenceslas-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Reginald-Jacques/Good-King-Wenceslas-lyrics/

http://www.lyricsmode.com/lyrics/R/Reginald_Jacques/Good_King_Wenceslas.html

http://metrolyrics.com/Good-King-Wenceslas-lyrics-Reginald-Jacques.html

https://genius.com/John-Rutter-All-Bells-in-Paradise-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/John-Rutter/All-Bells-in-Paradise-lyrics/

http://www.lyricsmode.com/lyrics/J/John_Rutter/All_Bells_in_Paradise.html

http://metrolyrics.com/All-Bells-in-Paradise-lyrics-John-Rutter.html

https://genius.com/Traditional-I-Saw-a-Maiden-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Traditional/I-Saw-a-Maiden-lyrics/

http://www.lyricsmode.com/lyrics/T/Traditional/I_Saw_a_Maiden.html

http://metrolyrics.com/I-Saw-a-Maiden-lyrics-Traditional.html

https://gen

http://metrolyrics.com/Es-ist-ein-Ros-entsprungen-lyrics-Michael-Praetorius.html

https://genius.com/Choir-of-Kings-College-Cambridge-Past-Three-OClock-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Choir-of-Kings-College-Cambridge/Past-Three-OClock-lyrics/

http://www.lyricsmode.com/lyrics/C/Choir_of_Kings_College_Cambridge/Past_Three_OClock.html

http://metrolyrics.com/Past-Three-OClock-lyrics-Choir-of-Kings-College-Cambridge.html

https://genius.com/Arvo-Part-Christmas-Lullaby-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Arvo-Part/Christmas-Lullaby-lyrics/

http://www.lyricsmode.com/lyrics/A/Arvo_Part/Christmas_Lullaby.html

http://metrolyrics.com/Christmas-Lullaby-lyrics-Arvo-Part.html

https://genius.com/William-Mathias-Sir-Christemas-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/William-Mathias/Sir-Christemas-lyrics/

http://www.lyricsmode.com/lyrics/W/William_Mathias/Sir_Christemas.html

http://metrolyric

http://songlyrics.com/Heinrich-Schutz/Geistliche-Chormusik-Ein-Kind-ist-uns-geboren-SWV-384-lyrics/

http://www.lyricsmode.com/lyrics/H/Heinrich_Schutz/Geistliche_Chormusik_Ein_Kind_ist_uns_geboren_SWV_384.html

http://metrolyrics.com/Geistliche-Chormusik-Ein-Kind-ist-uns-geboren-SWV-384-lyrics-Heinrich-Schutz.html

https://genius.com/Benjamin-Britten-Ceremony-of-Carols-Op28-Balulalow-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Benjamin-Britten/Ceremony-of-Carols-Op28-Balulalow-lyrics/

http://www.lyricsmode.com/lyrics/B/Benjamin_Britten/Ceremony_of_Carols_Op28_Balulalow.html

http://metrolyrics.com/Ceremony-of-Carols-Op28-Balulalow-lyrics-Benjamin-Britten.html

https://genius.com/Alessandro-Scarlatti-O-di-Betlemme-altera-poverta-ventu-I-Sinfonia-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Alessandro-Scarlatti/O-di-Betlemme-altera-poverta-ventu-I-Sinfonia-lyrics/

http://www.lyricsmode.com/lyrics/A/Alessandro_Scarlatti/O_di_Betlemme

https://genius.com/Rod-Stewart-Have-I-Told-You-Lately-lyrics
https://genius.com/Whitney-Houston-Where-Do-Broken-Hearts-Go-lyrics
https://genius.com/Jess-Glynne-Take-Me-Home-lyrics
https://genius.com/Michael-Schulte-You-Said-Youd-Grow-Old-With-Me-lyrics
https://genius.com/Julie-Bergan-Ill-Be-Fine-Somehow-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Julie-Bergan/Ill-Be-Fine-Somehow-lyrics/

http://www.lyricsmode.com/lyrics/J/Julie_Bergan/Ill_Be_Fine_Somehow.html

http://metrolyrics.com/Ill-Be-Fine-Somehow-lyrics-Julie-Bergan.html

https://genius.com/Richard-Marx-Right-Here-Waiting-lyrics
https://genius.com/LeAnn-Rimes-How-Do-I-Live-lyrics
https://genius.com/Demi-Lovato-Stone-Cold-lyrics
https://genius.com/Phil-Collins-Against-All-Odds-lyrics
'NoneType' object has no attribute 'find'
http://songlyrics.com/Phil-Collins/Against-All-Odds-lyrics/
https://genius.com/Ronan-Keating-If-Tomorrow-Never-Comes-lyrics
https://genius.com/Coldplay-Everglow-lyrics
https://genius

In [6]:
with open(LYRICS_FILE_NAME, 'w') as f:
    json.dump(lyrics_list, f)

In [7]:
len(lyrics_list)

5000